In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import to_datetime
from datetime import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
df = pd.read_csv('../modelo/LaLiga_Matches_1995-2021.csv', parse_dates=['Date'], dayfirst=True)

print(df.head())

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

    Season      Date    HomeTeam    AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR
0  1995-96  2/9/1995   La Coruna    Valencia     3     0   H   2.0   0.0   H
1  1995-96  2/9/1995    Sp Gijon    Albacete     3     0   H   3.0   0.0   H
2  1995-96  3/9/1995  Ath Bilbao   Santander     4     0   H   2.0   0.0   H
3  1995-96  3/9/1995  Ath Madrid    Sociedad     4     1   H   1.0   1.0   D
4  1995-96  3/9/1995       Celta  Compostela     0     1   A   0.0   0.0   D


In [2]:

def goles_a_favor_ultimos_cinco(equipo, fecha_init,df):
    #Comprueba que los últimos 5 partidos jugados por ese equipo pertenezcan a la temporada en cuestión
    #por lo que si le pedimos la predicción un 5 de septiembre solo tendrá en cuenta los partidos jugados hasta el 5 de sept pero a partir del 1 de agosto, 
    #es decir de la temporada nueva que estamos pidiendo
    fecha = pd.to_datetime(fecha_init)
    fecha_str = fecha.strftime("%d/%m/%Y")
    fecha = datetime.strptime(fecha_str, "%d/%m/%Y")

    year = int(fecha.strftime("%Y"))
    
    if year == 1999:
        if pd.to_datetime(fecha) >= pd.Timestamp("1999-08-01"):
            temporada = f"{year}-{int(str(year+1)[-4:])}"
        else:
            temporada = f"{year-1}-{int(str(year)[-2:])}"
    elif year == 2000:
        if pd.to_datetime(fecha) <= pd.Timestamp("2000-08-01"):
            temporada = f"{year-1}-{int(str(year)[-4:])}"
        else:
            temporada = f"{year}-{int(str(year+1)[-2:])}"
            
    elif fecha.strftime("%Y-%m-%d") >= f"{year}-08-01":
        temporada = f"{year}-{int(str(year+1)[-2:])}"
    else:
        temporada = f"{year-1}-{int(str(year)[-2:])}"
        
    if len(temporada) == 6:
        temporada = f"{temporada[0:4]}-0{temporada[-1]}"
        
    partidos_equipo = df[((df['HomeTeam'] == equipo) | (df['AwayTeam'] == equipo)) & (df['Date'] < fecha_init) & (df['Season'] == temporada)]
    partidos_ordenados = partidos_equipo.sort_values('Date', ascending=False)
    partidos_ultimos_cinco = partidos_ordenados[(partidos_ordenados['HomeTeam'] == equipo) | (partidos_ordenados['AwayTeam'] == equipo)].head(5)
    goles = 0
    for i, partido in partidos_ultimos_cinco.iterrows():
        if partido['HomeTeam'] == equipo:
            goles += partido['FTHG']
        elif partido['AwayTeam'] == equipo:
            goles += partido['FTAG']
    return goles
    
    

def goles_en_contra_ultimos_cinco(equipo, fecha_init,df):
    
    fecha = pd.to_datetime(fecha_init)
    fecha_str = fecha.strftime("%d/%m/%Y")
    fecha = datetime.strptime(fecha_str, "%d/%m/%Y")
    year = int(fecha.strftime("%Y"))    
    if year == 1999:
        if pd.to_datetime(fecha) >= pd.Timestamp("1999-08-01"):
            temporada = f"{year}-{int(str(year+1)[-4:])}"
        else:
            temporada = f"{year-1}-{int(str(year)[-2:])}"
    elif year == 2000:
        if pd.to_datetime(fecha) <= pd.Timestamp("2000-08-01"):
            temporada = f"{year-1}-{int(str(year)[-4:])}"
        else:
            temporada = f"{year}-{int(str(year+1)[-2:])}"
    elif fecha.strftime("%Y-%m-%d") >= f"{year}-08-01":
        temporada = f"{year}-{int(str(year+1)[-2:])}"
    else:
        temporada = f"{year-1}-{int(str(year)[-2:])}"
    
    if len(temporada) == 6:
        temporada = f"{temporada[0:4]}-0{temporada[-1]}"
        
    partidos_equipo = df[((df['HomeTeam'] == equipo) | (df['AwayTeam'] == equipo)) & (df['Date'] < fecha_init) & (df['Season'] == temporada)]
    partidos_ordenados = partidos_equipo.sort_values('Date', ascending=False)
    partidos_ultimos_cinco = partidos_ordenados[(partidos_ordenados['HomeTeam'] == equipo) | (partidos_ordenados['AwayTeam'] == equipo)].head(5)
    goles = 0
    for i, partido in partidos_ultimos_cinco.iterrows():
        if partido['HomeTeam'] == equipo:
            goles += partido['FTAG']
        elif partido['AwayTeam'] == equipo:
            goles += partido['FTHG']
    return goles


def goles_a_favor_descanso_ultimos_cinco(equipo, fecha_init,df):
    fecha = pd.to_datetime(fecha_init)
    fecha_str = fecha.strftime("%d/%m/%Y")
    fecha = datetime.strptime(fecha_str, "%d/%m/%Y")
    year = int(fecha.strftime("%Y"))    

    if year == 1999:
        if pd.to_datetime(fecha) >= pd.Timestamp("1999-08-01"):
            temporada = f"{year}-{int(str(year+1)[-4:])}"
        else:
            temporada = f"{year-1}-{int(str(year)[-2:])}"
    elif year == 2000:
        if pd.to_datetime(fecha) <= pd.Timestamp("2000-08-01"):
            temporada = f"{year-1}-{int(str(year)[-4:])}"
        else:
            temporada = f"{year}-{int(str(year+1)[-2:])}"
    elif fecha.strftime("%Y-%m-%d") >= f"{year}-08-01": 
        temporada = f"{year}-{int(str(year+1)[-2:])}"
    else:
        temporada = f"{year-1}-{int(str(year)[-2:])}"
    if len(temporada) == 6:
        temporada = f"{temporada[0:4]}-0{temporada[-1]}"
        
    partidos_equipo = df[((df['HomeTeam'] == equipo) | (df['AwayTeam'] == equipo)) & (df['Date'] < fecha_init) & (df['Season'] == temporada)]
    partidos_ordenados = partidos_equipo.sort_values('Date', ascending=False)
    partidos_ultimos_cinco = partidos_ordenados[(partidos_ordenados['HomeTeam'] == equipo) | (partidos_ordenados['AwayTeam'] == equipo)].head(5)
    goles = 0
    for i, partido in partidos_ultimos_cinco.iterrows():
        if partido['HomeTeam'] == equipo:
            goles += partido['HTHG']
        elif partido['AwayTeam'] == equipo:
            goles += partido['HTAG']
    return goles


def goles_en_contra_descanso_ultimos_cinco(equipo, fecha_init,df):
    fecha = pd.to_datetime(fecha_init)
    fecha_str = fecha.strftime("%d/%m/%Y")
    fecha = datetime.strptime(fecha_str, "%d/%m/%Y")
    year = int(fecha.strftime("%Y"))    

    if year == 1999:
        if pd.to_datetime(fecha) >= pd.Timestamp("1999-08-01"):
            temporada = f"{year}-{int(str(year+1)[-4:])}"
        else:
            temporada = f"{year-1}-{int(str(year)[-2:])}"
    elif year == 2000:
        if pd.to_datetime(fecha) <= pd.Timestamp("2000-08-01"):
            temporada = f"{year-1}-{int(str(year)[-4:])}"
        else:
            temporada = f"{year}-{int(str(year+1)[-2:])}"
    elif fecha.strftime("%Y-%m-%d") >= f"{year}-08-01": 
        temporada = f"{year}-{int(str(year+1)[-2:])}"
    else:
        temporada = f"{year-1}-{int(str(year)[-2:])}"
    if len(temporada) == 6:
        temporada = f"{temporada[0:4]}-0{temporada[-1]}"
        
    partidos_equipo = df[((df['HomeTeam'] == equipo) | (df['AwayTeam'] == equipo)) & (df['Date'] < fecha_init) & (df['Season'] == temporada)]
    partidos_ordenados = partidos_equipo.sort_values('Date', ascending=False)
    partidos_ultimos_cinco = partidos_ordenados[(partidos_ordenados['HomeTeam'] == equipo) | (partidos_ordenados['AwayTeam'] == equipo)].head(5)
    goles = 0
    for i, partido in partidos_ultimos_cinco.iterrows():
        if partido['HomeTeam'] == equipo:
            goles += partido['HTAG']
        elif partido['AwayTeam'] == equipo:
            goles += partido['HTHG']
    return goles


def puntos_ultimos_cinco(equipo, fecha_init,df):
    fecha = pd.to_datetime(fecha_init)
    fecha_str = fecha.strftime("%d/%m/%Y")
    fecha = datetime.strptime(fecha_str, "%d/%m/%Y")
    year = int(fecha.strftime("%Y"))    
    
    
    if year == 1999:
        if pd.to_datetime(fecha) >= pd.Timestamp("1999-08-01"):
            temporada = f"{year}-{int(str(year+1)[-4:])}"
        else:
            temporada = f"{year-1}-{int(str(year)[-2:])}"
    elif year == 2000:
        if pd.to_datetime(fecha) <= pd.Timestamp("2000-08-01"):
            temporada = f"{year-1}-{int(str(year)[-4:])}"
        else:
            temporada = f"{year}-{int(str(year+1)[-2:])}"
    elif fecha.strftime("%Y-%m-%d") >= f"{year}-08-01":
        temporada = f"{year}-{int(str(year+1)[-2:])}"
    else:
        temporada = f"{year-1}-{int(str(year)[-2:])}"
    if len(temporada) == 6:
        temporada = f"{temporada[0:4]}-0{temporada[-1]}"
        
    partidos_equipo = df[((df['HomeTeam'] == equipo) | (df['AwayTeam'] == equipo)) & (df['Date'] < fecha_init) & (df['Season'] == temporada)]
    partidos_ordenados = partidos_equipo.sort_values(['Date'], ascending=[False])
    partidos_ultimos_cinco = partidos_ordenados[(partidos_ordenados['HomeTeam'] == equipo) | (partidos_ordenados['AwayTeam'] == equipo)].head(5)
    puntos = 0
    for i, partido in partidos_ultimos_cinco.iterrows():
        if partido['HomeTeam'] == equipo:
            if(partido['FTR'] == 'H'):
                puntos += 3
            elif(partido['FTR'] == 'D'):
                puntos += 1
            elif(partido['FTR'] == 'A'):
                puntos += 0
        elif partido['AwayTeam'] == equipo:
            
            if(partido['FTR'] == 'A'):
                puntos += 3
            elif(partido['FTR'] == 'D'):
               
                puntos += 1
            elif(partido['FTR'] == 'H'):
               
                puntos += 0
            
    return puntos


def clasificacion_en_una_fecha(fecha_init,df):
    
    fecha = pd.to_datetime(fecha_init)
    fecha_str = fecha.strftime("%d/%m/%Y")
    fecha = datetime.strptime(fecha_str, "%d/%m/%Y")
    year = int(fecha.strftime("%Y"))        
    if year == 1999:
        if pd.to_datetime(fecha_init) >= pd.Timestamp("1999-08-01"):
            temporada = f"{year}-{int(str(year+1)[-4:])}"
        else:
            temporada = f"{year-1}-{int(str(year)[-2:])}"
    elif year == 2000:
        if pd.to_datetime(fecha_init) <= pd.Timestamp("2000-08-01"):
            temporada = f"{year-1}-{int(str(year)[-4:])}"
        else:
            temporada = f"{year}-{int(str(year+1)[-2:])}"
    elif pd.to_datetime(fecha_init) >= pd.Timestamp(f"{year}-08-01"):
        temporada = f"{year}-{int(str(year+1)[-2:])}"
    else:
        temporada = f"{year-1}-{int(str(year)[-2:])}"
    if len(temporada) == 6:
        temporada = f"{temporada[0:4]}-0{temporada[-1]}"
        
    partidos = df[(df['Date'] < fecha_init) & (df['Season'] == temporada)]
    puntos = {}
    for _, partido in partidos.iterrows():
        if partido['HomeTeam'] not in puntos:
            puntos[partido['HomeTeam']] = 0
        if partido['AwayTeam'] not in puntos:
            puntos[partido['AwayTeam']] = 0
        if partido['FTR'] == 'D':
            puntos[partido['HomeTeam']] += 1
            puntos[partido['AwayTeam']] += 1
        elif partido['FTR'] == 'H':
            puntos[partido['HomeTeam']] += 3
        else:
            puntos[partido['AwayTeam']] += 3
    
    clasificacion = sorted(puntos.items(), key=lambda x: x[1], reverse=True)
    return clasificacion


def posicion_equipo_en_una_fecha(equipo, fecha,df):
    clasificacion = clasificacion_en_una_fecha(fecha,df)
    posicion = [i+1 for i, (eq, pts) in enumerate(clasificacion) if eq == equipo]
    if posicion:
        return posicion[0]
    else:
        return -1
    
    



In [3]:
columnas = ['Temporada', 'Fecha', 'EquipoLocal', 'GolesFavL5', 'GolesConL5', 'PuntosL5', 'PosicionLocal', 'GolesFav1L5', 'GolesCont1L5', 'EquipoVisitante', ' GolesFavV5', 'GolesConV5', 'PuntosV5', 'PosicionVisitante', 'GolesFav1V5', 'GolesCon1V5']
df_nuevo = pd.DataFrame(columns=columnas)
df_nuevo.head()

,Temporada,Fecha,EquipoLocal,GolesFavL5,GolesConL5,PuntosL5,PosicionLocal,GolesFav1L5,GolesCont1L5,EquipoVisitante,GolesFavV5,GolesConV5,PuntosV5,PosicionVisitante,GolesFav1V5,GolesCon1V5


In [4]:

partidos = df

datos_partidos = []
for _, partido in partidos.iterrows():
    temporada = partido['Season']
    fecha = partido['Date']
    ganador = partido['FTR']
    goles_local = partido['FTHG']
    goles_visitante = partido['FTAG']
    goles_local_primera_parte = partido['HTHG']
    goles_visitante_primera_parte = partido['HTAG']
    
    # EQUIPO LOCAL
    equipo_local = partido['HomeTeam']
    goles_a_favor_local_ultimos_5 = goles_a_favor_ultimos_cinco(equipo_local, fecha, df)
    goles_en_contra_local_ultimos_5 = goles_en_contra_ultimos_cinco(equipo_local, fecha,df)
    puntos_local_ultimos_5 = puntos_ultimos_cinco( equipo_local, fecha,df)
    posicion_local = posicion_equipo_en_una_fecha(equipo_local, fecha,df) 
    goles_a_favor_primera_parte_local_ultimos_5 = goles_a_favor_descanso_ultimos_cinco(equipo_local, fecha,df)
    goles_en_contra_primera_parte_local_ultimos_5 = goles_en_contra_descanso_ultimos_cinco(equipo_local, fecha,df)
 
    
    # EQUIPO VISITANTE
    equipo_visitante = partido['AwayTeam']
    goles_a_favor_visitante_ultimos_5 = goles_a_favor_ultimos_cinco(equipo_visitante, fecha,df)
    goles_en_contra_visitante_ultimos_5 = goles_en_contra_ultimos_cinco(equipo_visitante, fecha,df)
    puntos_visitante_ultimos_5 = puntos_ultimos_cinco( equipo_visitante, fecha,df)
    posicion_visitante = posicion_equipo_en_una_fecha(equipo_visitante, fecha,df)
    goles_a_favor_primera_parte_visitante_ultimos_5 = goles_a_favor_descanso_ultimos_cinco(equipo_visitante, fecha,df)
    goles_en_contra_primera_parte_visitante_ultimos_5 = goles_en_contra_descanso_ultimos_cinco(equipo_visitante, fecha,df)

    nueva_fila = {'Temporada': temporada, 'Fecha': fecha,'Ganador': ganador, 'Goles Local': goles_local,'Goles Visitante': goles_visitante, 'Goles Local primera parte': goles_local_primera_parte, 'Goles Visitante primera parte': goles_visitante_primera_parte, 'EquipoLocal': equipo_local, 'GolesFavL5': goles_a_favor_local_ultimos_5, 'GolesConL5': goles_en_contra_local_ultimos_5, 'PuntosL5': puntos_local_ultimos_5, 'PosicionLocal': posicion_local, 'GolesFav1L5': goles_a_favor_primera_parte_local_ultimos_5, 'GolesCont1L5': goles_en_contra_primera_parte_local_ultimos_5,
                  'EquipoVisitante': equipo_visitante, ' GolesFavV5': goles_a_favor_visitante_ultimos_5, 'GolesConV5': goles_en_contra_visitante_ultimos_5, 'PuntosV5': puntos_visitante_ultimos_5, 'PosicionVisitante': posicion_visitante, 'GolesFav1V5': goles_a_favor_primera_parte_visitante_ultimos_5, 'GolesCon1V5': goles_en_contra_primera_parte_visitante_ultimos_5}
    
    nueva_fila = pd.DataFrame(nueva_fila, index=[0])
    df_nuevo = pd.concat([df_nuevo, nueva_fila], ignore_index=True)
df_nuevo.to_csv('partidos_caracteristicas.csv', index=False)

df_nuevo.head()


/var/folders/tr/vj2kx_4d6r9dcnv0qjgxcq580000gn/T/ipykernel_26045/2390736676.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  fecha = pd.to_datetime(fecha_init)
/var/folders/tr/vj2kx_4d6r9dcnv0qjgxcq580000gn/T/ipykernel_26045/2390736676.py:45: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  fecha = pd.to_datetime(fecha_init)
/var/folders/tr/vj2kx_4d6r9dcnv0qjgxcq580000gn/T/ipykernel_26045/2390736676.py:150: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  fecha = pd.to_datetime(fecha_init)
/var/folders/tr/vj2kx_4d6r9dcnv0qjgxcq580000gn/T/ipykernel_26045/2390736676.py:201: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the de

,Temporada,Fecha,EquipoLocal,GolesFavL5,GolesConL5,PuntosL5,PosicionLocal,GolesFav1L5,GolesCont1L5,EquipoVisitante,...,GolesConV5,PuntosV5,PosicionVisitante,GolesFav1V5,GolesCon1V5,Ganador,Goles Local,Goles Visitante,Goles Local primera parte,Goles Visitante primera parte
0,1995-96,2/9/1995,La Coruna,0,0,0,-1,0,0,Valencia,...,0,0,-1,0,0,H,3.0,0.0,2.0,0.0
1,1995-96,2/9/1995,Sp Gijon,0,0,0,-1,0,0,Albacete,...,0,0,-1,0,0,H,3.0,0.0,3.0,0.0
2,1995-96,3/9/1995,Ath Bilbao,0,0,0,-1,0,0,Santander,...,0,0,-1,0,0,H,4.0,0.0,2.0,0.0
3,1995-96,3/9/1995,Ath Madrid,0,0,0,-1,0,0,Sociedad,...,0,0,-1,0,0,H,4.0,1.0,1.0,1.0
4,1995-96,3/9/1995,Celta,0,0,0,-1,0,0,Compostela,...,0,0,-1,0,0,A,0.0,1.0,0.0,0.0


In [14]:
df_nuevo = pd.read_csv('../modelo/partidos_caracteristicas.csv', parse_dates=['Fecha'], dayfirst=True)
df = pd.read_csv('../modelo/LaLiga_Matches_1995-2021.csv', parse_dates=['Date'], dayfirst=True)
# Teniendo en cuenta que cada temporada tiene 380 partidos (50 primeras filas de cada temporada, es decir las primeras 5 jornadas)
filas_a_eliminar = []
for i in range(25):
    inicio_temporada = i * 380
    fin_temporada = inicio_temporada + 50
    filas_temporada = list(range(inicio_temporada, fin_temporada))
    filas_a_eliminar.extend(filas_temporada)
    
#Elimina los partidos de la temporada 20-21 del conjunto de datos de entrenamiento 
#ya que es la temporada que vamos a predecir al final.
partidos_2020_2021 = df[(df['Season'] == '2020-21')]

filas_a_eliminar.extend(partidos_2020_2021.index)
df_nuevo = df_nuevo.drop(filas_a_eliminar)
df_nuevo.head()
df_nuevo.to_csv('partidos_caracteristicas_hasta_2019-20.csv', index=False)


In [15]:
df_nuevo = pd.read_csv('../modelo/partidos_caracteristicas.csv', parse_dates=['Fecha'], dayfirst=True)

partidos_a_predecir = df_nuevo[(df_nuevo['Temporada'] == '2020-21')]

partidos_a_predecir.to_csv('partidos-2020-21.csv', index = False)

In [16]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

#Dataset hasta 2019-20
df = pd.read_csv("../modelo/partidos_caracteristicas_hasta_2019-20.csv") 

df['Ganador'] = df['Ganador'].map({'D': 0, 'H': 1, 'A': 2})  # Convertir las etiquetas de Ganador a tipo float

# Entrenar el modelo con todos los partidos anteriores
X = df[['GolesFavL5', 'GolesConL5', 'PuntosL5', 'PosicionLocal', ' GolesFavV5', 'GolesConV5', 'PuntosV5', 'PosicionVisitante']]
y = df['Ganador']


y = np.array(y)

model = LogisticRegression(solver='liblinear')
model.fit(X, y)
accuracy = model.score(X, y)
print(accuracy)


# Filtrar los partidos de la temporada 2020-2021
season_2020_2021 = pd.read_csv("../modelo/partidos-2020-21.csv") 
# Seleccionar las columnas de interés
X_pred = season_2020_2021[['GolesFavL5', 'GolesConL5', 'PuntosL5', 'PosicionLocal', ' GolesFavV5', 'GolesConV5', 'PuntosV5', 'PosicionVisitante']]
pred = model.predict_proba(X_pred)



# Crear un nuevo DataFrame con las columnas deseadas
output_df = pd.DataFrame({
    'Fecha': season_2020_2021['Fecha'],
    'EquipoLocal': season_2020_2021['EquipoLocal'],
    'EquipoVisitante': season_2020_2021['EquipoVisitante'],
    'Prob_Victoria_Local': (pred[:, 1] * 100).round(2).astype(int),  # Probabilidad de victoria del local
    'Prob_Empate': (pred[:, 0] * 100).round(2).astype(int),          # Probabilidad de empate
    'Prob_Victoria_Visitante': (pred[:, 2] * 100).round(2).astype(int)  # Probabilidad de victoria del visitante
})

output_df.head()

0.49512716900404086


,Fecha,EquipoLocal,EquipoVisitante,Prob_Victoria_Local,Prob_Empate,Prob_Victoria_Visitante
0,12/9/2020,Eibar,Celta,39,27,32
1,12/9/2020,Granada,Ath Bilbao,59,24,16
2,12/9/2020,Cadiz,Osasuna,52,25,21
3,13/09/2020,Alaves,Betis,33,27,39
4,13/09/2020,Valladolid,Sociedad,41,26,31


In [17]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

#Dataset hasta 2019-20
df = pd.read_csv("../modelo/partidos_caracteristicas_hasta_2019-20.csv") 

df['Ganador'] = df['Ganador'].map({'D': 0, 'H': 1, 'A': 2})  # Convertir las etiquetas de Ganador a tipo float

# Entrenar el modelo con todos los partidos anteriores
X = df[['GolesFavL5', 'GolesConL5', 'PuntosL5', 'PosicionLocal', ' GolesFavV5', 'GolesConV5', 'PuntosV5', 'PosicionVisitante']]
y = df['Ganador']


y = np.array(y)

model = LogisticRegression(solver='liblinear')
model.fit(X, y)
accuracy = model.score(X, y)
print(accuracy)


# Filtrar los partidos de la temporada 2020-2021
season_2020_2021 = pd.read_csv("../modelo/partidos-2020-21.csv") 
# Seleccionar las columnas de interés
X_pred = season_2020_2021[['GolesFavL5', 'GolesConL5', 'PuntosL5', 'PosicionLocal', ' GolesFavV5', 'GolesConV5', 'PuntosV5', 'PosicionVisitante']]
pred = model.predict_proba(X_pred)



# Crear un nuevo DataFrame con las columnas deseadas
output_df = pd.DataFrame({
    'Fecha': season_2020_2021['Fecha'],
    'EquipoLocal': season_2020_2021['EquipoLocal'],
    'EquipoVisitante': season_2020_2021['EquipoVisitante'],
    'Prob_Victoria_Local': (pred[:, 1] * 100).round(2).astype(int),  # Probabilidad de victoria del local
    'Prob_Empate': (pred[:, 0] * 100).round(2).astype(int),          # Probabilidad de empate
    'Prob_Victoria_Visitante': (pred[:, 2] * 100).round(2).astype(int)  # Probabilidad de victoria del visitante
})

output_df.head()


0.49512716900404086


,Fecha,EquipoLocal,EquipoVisitante,Prob_Victoria_Local,Prob_Empate,Prob_Victoria_Visitante
0,12/9/2020,Eibar,Celta,39,27,32
1,12/9/2020,Granada,Ath Bilbao,59,24,16
2,12/9/2020,Cadiz,Osasuna,52,25,21
3,13/09/2020,Alaves,Betis,33,27,39
4,13/09/2020,Valladolid,Sociedad,41,26,31


In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error


df = pd.read_csv("../modelo/partidos_caracteristicas_hasta_2019-20.csv")

X = df[['GolesFavL5','GolesConL5', 'PuntosL5', 'PosicionLocal', ' GolesFavV5', 'GolesConV5', 'PuntosV5', 'PosicionVisitante']]
y = df['Goles Local']
y2 = df['Goles Visitante']



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("MSE goles local LinearRegression:", mse)




X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y2, test_size=0.2, random_state=42)

regressor2 = LinearRegression()
regressor2.fit(X_train2, y_train2)
y_pred2 = regressor2.predict(X_test2)

mse = mean_squared_error(y_test2, y_pred2)
print("")
print("MSE goles visitante LinearRegression:", mse)

season_2020_2021 = pd.read_csv('../modelo/partidos-2020-21.csv')

Z_pred = season_2020_2021[['GolesFavL5', 'GolesConL5', 'PuntosL5', 'PosicionLocal', ' GolesFavV5', 'GolesConV5', 'PuntosV5', 'PosicionVisitante']]

y_pred_local = np.round(regressor.predict(Z_pred)).astype(int)
y_pred_visitante = np.round(regressor2.predict(Z_pred)).astype(int)
output_df['golesLocalFinal'] = y_pred_local
output_df['golesVisitanteFinal'] = y_pred_visitante
output_df.head()

MSE goles local LinearRegression: 1.6667299017687798

MSE goles visitante LinearRegression: 1.1899386573955681


,Fecha,EquipoLocal,EquipoVisitante,Prob_Victoria_Local,Prob_Empate,Prob_Victoria_Visitante,golesLocalFinal,golesVisitanteFinal
0,12/9/2020,Eibar,Celta,39,27,32,1,1
1,12/9/2020,Granada,Ath Bilbao,59,24,16,2,1
2,12/9/2020,Cadiz,Osasuna,52,25,21,2,1
3,13/09/2020,Alaves,Betis,33,27,39,1,1
4,13/09/2020,Valladolid,Sociedad,41,26,31,1,1


In [19]:
df = pd.read_csv("../modelo/partidos_caracteristicas_hasta_2019-20.csv")
df = df.dropna() #elimina filas con valores nulos
X = df[['GolesFav1L5','GolesCont1L5', 'PuntosL5', 'PosicionLocal', 'GolesFav1V5', 'GolesCon1V5', 'PuntosV5', 'PosicionVisitante']]
y = df['Goles Local primera parte']
y2 = df['Goles Visitante primera parte']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("MSE goles local 1parte:", mse)

X_train, X_test, y_train2, y_test2 = train_test_split(X, y2, test_size=0.2, random_state=42)

regressor2 = LinearRegression()
regressor2.fit(X_train, y_train2)
y_pred2 = regressor2.predict(X_test)

mse = mean_squared_error(y_test2, y_pred2)
print("MSE goles visitante 1parte:", mse)

season_2020_2021 = pd.read_csv('../modelo/partidos-2020-21.csv')
Z_pred = season_2020_2021[['GolesFav1L5','GolesCont1L5', 'PuntosL5', 'PosicionLocal', 'GolesFav1V5', 'GolesCon1V5', 'PuntosV5', 'PosicionVisitante']]
y_pred_local = np.round(regressor.predict(Z_pred),).astype(int)
y_pred_visitante = np.round(regressor2.predict(Z_pred)).astype(int)
output_df['golesLocalDescanso'] = y_pred_local
output_df['golesVisitanteDescanso'] = y_pred_visitante
output_df['golesLocalReal'] = season_2020_2021['Goles Local']
output_df['golesVisitanteReal'] = season_2020_2021['Goles Visitante']
output_df['Ganador'] = season_2020_2021['Ganador']

output_df.head()


MSE goles local 1parte: 0.7461203471099365
MSE goles visitante 1parte: 0.502594817147105


,Fecha,EquipoLocal,EquipoVisitante,Prob_Victoria_Local,Prob_Empate,Prob_Victoria_Visitante,golesLocalFinal,golesVisitanteFinal,golesLocalDescanso,golesVisitanteDescanso,golesLocalReal,golesVisitanteReal,Ganador
0,12/9/2020,Eibar,Celta,39,27,32,1,1,1,1,0.0,0.0,D
1,12/9/2020,Granada,Ath Bilbao,59,24,16,2,1,1,0,2.0,0.0,H
2,12/9/2020,Cadiz,Osasuna,52,25,21,2,1,1,0,0.0,2.0,A
3,13/09/2020,Alaves,Betis,33,27,39,1,1,0,1,0.0,1.0,A
4,13/09/2020,Valladolid,Sociedad,41,26,31,1,1,1,0,1.0,1.0,D


In [20]:
output_df.to_csv('predicciones-temporada-2020-21.csv', index=False)


